In [ ]:
%matplotlib inline


from beras.data_utils import HDF5Tensor
import matplotlib.pyplot as plt
import numpy as np
from deepdecoder.utils import masks, visualise_tiles, zip_visualise_tiles, np_binary_mask
from deepdecoder.dcgan_train import generator, discriminator, dcmogan
from beesgrid import MASK, CONFIG_ROTS, CONFIG_RADIUS, CONFIG_CENTER, TAG_SIZE
from keras.optimizers import SGD, Adam, RMSprop
import importlib
import h5py
import pylab
import time

pylab.rcParams['figure.figsize'] = (18, 18)

In [ ]:
gan, grid_loss_weight = dcmogan(generator, discriminator)

In [ ]:
start = time.time()
gan.compile()
print("Done Compiling in {}s".format(time.time() - start))

In [ ]:
tags_fname = '/home/beesbook/beesbook-data/season_2015_tags/tags.hdf5'
h5 = h5py.File(tags_fname)

In [ ]:
batch_size = 128
nb_tags = h5['tags'].shape[0]
nb_tags = (nb_tags // batch_size)*batch_size

In [ ]:
tags = HDF5Tensor(tags_fname, 'tags', 0, nb_tags)

In [ ]:
def grid_config(grid_bits, grid_rot_pos_radius):
    angles = grid_rot_pos_radius[:, CONFIG_ROTS]
    sin = np.sin(angles)
    cos = np.cos(angles)
    r = grid_rot_pos_radius[:, (CONFIG_RADIUS,)] / 25. - 1
    xy = grid_rot_pos_radius[:, CONFIG_CENTER] / (TAG_SIZE/2) - 1
    configs = np.concatenate([grid_bits, sin, cos, xy, r], axis=1)
    return np.random.uniform(-1, 1, configs.shape)

In [ ]:
grid_loss_weight.set_value(0)

In [ ]:
epoch_size = 150 * batch_size
begin_grid_loss = 300
for i, (masks_idx, mask_bits, mask_grids) in enumerate(masks(epoch_size, scales=[1])):
    print(i)
    if i == begin_grid_loss:
        gan.gan.save("models/mogan_begin_grid_loss/")
        grid_loss_weight.set_value(np.cast[np.float32](0.1))
    if begin_grid_loss < i < begin_grid_loss + 49:
        grid_loss_weight.set_value(np.cast[np.float32](grid_loss_weight.get_value() + 0.05))
        
    config = grid_config(mask_bits, mask_grids)
    ti = i % nb_tags
    gan.mulit_objectives.fit([tags[ti:ti+epoch_size], masks_idx, config], verbose=1, nb_epoch=1)
    if i % 15 == 0 or (i < 16 and i % 1 == 0): 
        vis_idx, vis_bits, vis_grids = next(masks(batch_size-1, scales=[1]))
        vis_config = grid_config(vis_bits, vis_grids)
        zip_visualise_tiles(np_binary_mask(vis_idx),
                            gan.gan.generate(conditionals=[vis_config]))
    if i % 100 == 0 and i != 0:
        gan.gan.save("models/mogan_begin_grid_loss_{}/".format(i))

